In [2]:
import pandas as pd
import numpy as np

import torch
from transformers import AutoModelForMaskedLM, AutoTokenizer
from splade.models.transformer_rep import Splade

In [4]:
dataset = pd.read_json("./train.json")
dev = pd.read_json("./dev.json")
test = pd.read_json("./test.json")

h_train = dataset['History'].to_numpy()
q_train = dataset['Question'].to_numpy()
rw_train = dataset['Rewrite'].to_numpy()

h_dev = dev['History'].to_numpy()
q_dev = dev['Question'].to_numpy()
rw_dev = dev['Rewrite'].to_numpy()

h_test = test['History'].to_numpy()
q_test = test['Question'].to_numpy()
rw_test = test['Rewrite'].to_numpy()

In [6]:
model_type_or_dir = "naver/splade-cocondenser-ensembledistil"

# loading model and tokenizer

model = Splade(model_type_or_dir, agg="max")
model.eval()
tokenizer = AutoTokenizer.from_pretrained(model_type_or_dir)
reverse_voc = {v: k for k, v in tokenizer.vocab.items()}

Downloading:   0%|          | 0.00/670 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
for doc in q_train:
    with torch.no_grad():
        doc_rep = model(d_kwargs=tokenizer(doc, return_tensors="pt"))["d_rep"].squeeze()
        
# get the number of non-zero dimensions in the rep:
col = torch.nonzero(doc_rep).squeeze().cpu().tolist()
print("number of actual dimensions: ", len(col))